In [172]:
# import needed packages
import pandas as pd
import requests
from bs4 import BeautifulSoup
import math
import re
import time

In [173]:
# get id function
def get_id(term):
    id_list = []
    API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
    url_search = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200"
    info = BeautifulSoup(requests.get(url_search).text, 'lxml')
    
    term_count = int(info.find('count').text)
    id_list += [single_term.text for single_term in info.find_all('id')]
    
    for page in range(1, math.ceil(term_count/200)):
        page_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=gds&api_key={API_KEY}&term={term}&retmax=200&retstart={page*200}"
        page_info = BeautifulSoup(requests.get(page_url).text, 'lxml')
        id_list += [single_term.text for single_term in page_info.find_all('id')]
    
    return id_list, term_count


In [178]:
term = "(PBMC)+AND+(single-cell)"
id_list, term_count = get_id(term=term)

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [179]:
len(id_list)

7151

In [180]:
# check result
print(len(set(id_list)) == term_count)
print(term_count)

True
7151


In [181]:
# reserve result
with open('/home/ouyangkang/Learning/Python/other_thing/term_search_id_result_11_5.txt', 'w') as f:
    f.writelines("\n".join(id_list))

In [2]:
with open('/home/ouyangkang/Learning/Python/other_thing/term_search_id_result_11_5.txt', 'r') as f:
    id_list = [i.strip('\n') for i in f.readlines()]

In [325]:
# define a function to get detailed info
def id_detailed(id_list): # the max capacity of id list is 20
    API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
    return_list = []
    
    for page in range(math.ceil(len(id_list)/20)):
        id_search = ",".join(id_list[page*20:(page+1)*20])
        headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
                       "Accept-Encoding": "utf-8"}
        id_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key={API_KEY}&id={id_search}&rettype=summary"
        id_info = BeautifulSoup(requests.get(id_url, headers=headers).text, 'html.parser')

        # Title
        title = [i.text for i in id_info.find_all('item', {'name': 'title'})]
        # Summary
        summary = [i.text for i in id_info.find_all('item', {'name': 'summary'})]
        # GSE
        accession_pattern = re.compile(r'</id>\n<item name="Accession" type="String">([^<]+)</item>')
        gse = [re.search(accession_pattern, str(id_info).split('<docsum>')[i]).group(1) for i in range(1, len(str(id_info).split('<docsum>')))]
        # Taxon
        taxon = [i.text for i in id_info.find_all('item', {'name': 'taxon'})]
        # sequencing type
        seq_type = [i.text for i in id_info.find_all('item', {'name': 'gdsType'})]
        # Publish date
        p_date = [i.text for i in id_info.find_all('item', {'name': 'PDAT'})]
        # not each batch urls are 20
        for num in range(len(id_list[page*20:(page+1)*20])):
            return_list.append([title[num], summary[num], gse[num], taxon[num], seq_type[num], p_date[num]])
        # print(f"the {page*20}-{(page+1)*20} ids have finished")
        # print(page)


    return return_list

In [321]:
page = 10
API_KEY = "59cbd6ff6b3ce5d506f0eeb69e4e09362409"
id_search = ",".join(id_list[page*20:(page+1)*20])
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
                "Accept-Encoding": "utf-8"}
id_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi?db=gds&api_key={API_KEY}&id={id_search}&rettype=summary"
id_info = BeautifulSoup(requests.get(id_url, headers=headers).text, 'html.parser')

/home/ouyangkang/software/anaconda3/envs/ML/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [326]:
result_info = id_detailed(id_list=id_list)

In [327]:
# check length of each element
for i in range(len(result_info)):
    if len(result_info[i]) != 6:
        print(result_info[i])

print(len(result_info))

7151


In [349]:
# filter non-homo sapiens data and store result
no_filtering_result = pd.DataFrame(result_info, columns=['Title', 'Summary', 'GSE', 'Taxon', 'Sequencing_type', 'Publish_date'])
# avoid , impact csv format
no_filtering_result['Title'] = no_filtering_result['Title'].str.replace(',', '__')
no_filtering_result['Summary'] = no_filtering_result['Summary'].str.replace(',', '__')
# filtering
filtering_result = no_filtering_result[no_filtering_result['Taxon'].str.contains('Homo')]
filtering_result.to_csv('/home/ouyangkang/Learning/Python/other_thing/GEO_filtering_result_11_5.csv', index=False, encoding='UTF-8')

In [26]:
filtering_result = pd.read_csv('/home/ouyangkang/Learning/Python/other_thing/GEO_filtering_result_11_5.csv')

In [352]:
filtering_result.head()

,Title,Summary,GSE,Taxon,Sequencing_type,Publish_date
0,Factors contributing to Chimeric Antigen Recep...,Immune cell populations are composed of multip...,GSE187438,Homo sapiens,Expression profiling by high throughput sequen...,2023/11/02
1,Pathophysiology of androgen-associated endothe...,Abundant evidence suggests that women with pol...,GSE246456,Homo sapiens,Expression profiling by high throughput sequen...,2023/11/01
2,Immune landscape of PBMC based on single-cell ...,Although an increase in studies has revealed t...,GSE233315,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/31
3,Gene expression profile at single cell level o...,We used single cell RNA sequencing (scRNAseq) ...,GSE229626,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/31
4,Gene expression profiles of peripheral blood m...,An autoimmune B cell origin for childhood idio...,GSE233276,Homo sapiens,Expression profiling by high throughput sequen...,2023/10/29


In [361]:
print(len(set(filtering_result['GSE'])))
ges_list = list(set(filtering_result.GSE.tolist()))
set(filtering_result["GSE"].tolist()) - (set(filtering_result[filtering_result['GSE'].str.contains("GSM")]["GSE"].tolist()) | set(filtering_result[filtering_result['GSE'].str.contains("GSE")]["GSE"].tolist()))

6968


{'GPL96'}

In [164]:
def get_ges_info(ges_list):
    result_list = []
    for ges in ges_list:
        try:
            ges_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={ges}"
            headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
                       "Accept-Encoding": "utf-8"}
            ges_info = BeautifulSoup(requests.get(ges_url, headers=headers,timeout=(30, 10)).text, 'html.parser')

            info_pattern = re.compile(r'<td style="text-align: justify">(.*)</td>')
            getting_info = re.findall(info_pattern, str(ges_info))
            # title
            title_info = getting_info[0]
            # summary
            summary_info = getting_info[1].strip('<br/>')
            # over design
            design_info = getting_info[2].strip('<br/>')
            # organization name
            org_name = getting_info[3].strip('<br/>')
            # city
            city = getting_info[-3]
            # country
            country = getting_info[-1]
            # Experiment type
            exp_pattern = re.compile(r'<td>(.*)<br/></td>')
            exp_type = re.findall(exp_pattern, str(ges_info))[0]
            # contributor
            contributor = ";".join([i.text for i in ges_info.find_all('a', href=lambda value: value and '/pubmed/?term' in value)])
            # pmid
            try:
                pmid = ges_info.find('span', {'class': 'pubmed_id'}).text
            except:
                pmid = None
            # submission time and last update time
            time_pattern = re.compile(r'<td>([A-Z][a-z]+ \d{1,2}, \d{4})</td>')
            # first time is submission date, second last update date
            sub_time = re.findall(time_pattern, str(ges_info))[0]
            last_update_time = re.findall(time_pattern, str(ges_info))[1] 
            # platform
            platform = ges_info.find_all('td', {'valign':'top'})[1].text
            # sample
            gsm_list = []
            for i in ges_info.find_all('td', {'valign':'top'}):
                gsm = i.find('a', href=lambda value: value and '/geo/query/acc.cgi?acc' in value)
                if "GSM" in str(gsm):
                    gsm_list.append(gsm.get('href').strip('/geo/query/acc.cgi?acc='))
            sample_info = ";".join(gsm_list)
            # status
            statu_pattern = re.compile(r'<td>Public on (.*)</td>')
            statu = re.findall(statu_pattern, str(ges_info))[0]

            # "GSE", "Title", "PMID","Summary", "Design", "Experiment_type", "Platform", "Contributors", "Organization_name", "Country", "City", "Sample", "Status", "Submission_date", "Last_update_date"
            result_list.append([ges, title_info, pmid, summary_info,design_info, exp_type, platform, contributor, org_name, country, city, sample_info, statu, sub_time, last_update_time])
            time.sleep(5)
        except:
            print(ges)
    return result_list

In [165]:
gse_result = get_ges_info(ges_list)

GSE165238
GSE132048
GSE144744
GSE165816


In [167]:
check_ges = 'GSE165238'
ges_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={check_ges}"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
           "Accept-Encoding": "utf-8"}
ges_info = BeautifulSoup(requests.get(ges_url, headers=headers, timeout=(30, 10)).text, 'html.parser')

In [168]:
gsm_list = []
for i in ges_info.find_all('td', {'valign':'top'}):
    gsm = i.find('a', href=lambda value: value and '/geo/query/acc.cgi?acc' in value)
    if "GSM" in str(gsm):
        gsm_list.append(gsm.get('href').strip('/geo/query/acc.cgi?acc='))
sample_info = ";".join(gsm_list)
sample_info

'GSM5028734;GSM5028735;GSM5028736;GSM5028737;GSM5028738;GSM5028739;GSM5028740;GSM5028741;GSM5028742;GSM5028743;GSM5028744;GSM5028745;GSM5028746;GSM5028747;GSM5028748;GSM5028749'

In [170]:
statu_pattern = re.compile(r'<td>Public on (.*)</td>')
statu = re.findall(statu_pattern, str(ges_info))[0]
statu

'May 25, 2021'

In [166]:
result_print = pd.DataFrame(gse_result, columns=["GSE", "Title", "PMID","Summary", "Design", "Experiment_type", "Platform", "Contributors", "Organization_name", "Country", "City", "Sample", "Status", "Submission_date", "Last_update_date"])
result_print.to_csv('/home/ouyangkang/Learning/Python/other_thing/decoding_result.csv', index=None, encoding='utf-8')

In [163]:
result_print.head()

,GSE,Title,PMID,Summary,Design,Experiment_type,Platform,Contributors,Organization_name,Country,City,Sample,Status,Submission_date,Last_update_date
0,GSE165238,Tumor-associated hematopoietic stem and progen...,34162860,Brain tumors are typically immunosuppressive a...,Examination of single-cell transcriptomics fro...,Expression profiling by high throughput sequen...,Illumina NovaSeq 6000 (Homo sapiens),Kebir S;Cima I;Dobersalske C,DKFZ-Division Translational Neurooncology at t...,Germany,Essen,GSM5028734;GSM5028735;GSM5028736;GSM5028737;GS...,"May 25, 2021","Jan 21, 2021","Jul 14, 2021"
1,GSE132048,Linking Indirect Effects of Cytomegalovirus to...,32494628,Purpose: To use next-generation sequencing (NG...,Methods: mRNA profiles of human monocytes iso...,Expression profiling by high throughput sequen...,NextSeq 550 (Homo sapiens),Sen P;Wilkie AR;Ji F;Taylor IJ;Velasquez-Palaf...,Massachusetts General Hospital,USA,Massachusetts,GSM3839716;GSM3839717;GSM3839718;GSM3839719;GS...,"Apr 30, 2020","May 31, 2019","Aug 17, 2020"
2,GSE227952,Ferric citrate and apo-transferrin enable eryt...,37626061,We describe a simple and reproducible culture ...,Single-cell RNA sequencing was performed using...,Expression profiling by high throughput sequen...,HiSeq X Ten (Homo sapiens),Jeon S;Koh H;Han A;Kim J;Lee S;Lee J;Im S;Yoon...,82-43-240-6312,South Korea,Chungbuk,GSM7111359,"Aug 31, 2023","Mar 22, 2023","Sep 01, 2023"
3,GSE216883,Single cell transcriptome and surface protein ...,36787119,"We unveiled the characteristics of pTh17s, whi...",We performed transcriptome and cytometric anal...,Expression profiling by high throughput sequen...,HiSeq X Ten (Homo sapiens),Yi K;Jo S;Song W;Lee JS;Lee EY;Kim T,01096983470,South Korea,Daejeon,GSM6697222;GSM6697223;GSM6697224;GSM6697225;GS...,"Feb 23, 2023","Oct 31, 2022","Feb 24, 2023"
4,GSE218664,Early response evaluation by single cell signa...,36611026,Aberrant pro-survival signaling is a hallmark ...,In this study we investigate the singlaing res...,Expression profiling by high throughput sequen...,Illumina HiSeq 4000 (Homo sapiens),Tore Gjertsen B;Sjo Tislevoll B;Srivastava A;H...,92689503,Norway,Bergen,GSM6754319;GSM6754320;GSM6754321;GSM6754322;GS...,"Dec 02, 2022","Nov 23, 2022","Jan 25, 2023"


In [33]:
ges_list = list(set(filtering_result.GSE.tolist()))
ges_url = f"https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc={ges_list[0]}"
ges_info = BeautifulSoup(requests.get(ges_url, timeout=(5, 5)).text, 'html.parser')

In [34]:
ges_info

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">

<html>
<head>
<style type="text/css">
      a { text-decoration: none; }
	.bordergc {background-color: #6699CC;}
	.bordergd {background-color: #B6C7E5;}
	.borderge {background-color: #EEF3FB;}
	.bordergf {background-color: #FFFFFF;}
	.bordergg {background-color: #CCCCCC;}
      .small8b { font-size:8pt;
                font-family: ariel,helvetica,sans-serif;
                color:#6633cc;
              }
      .small8db { font-size:8pt;
                font-family: ariel,helvetica,sans-serif;
                color:#4411aa;
              }

    </style>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="NCBI GEO Gene Expression Omnibus microarray oligonucleotide array SAGE" name="keywords"/>
<meta content="NCBI's Gene Expression Omnibus (GEO) is a public archive and resource for gene expression data." name="description"/>
<meta content="geo" name="ncbi_app"/>
<meta content="full" name="ncb

In [120]:
# status
partern = re.compile(r'<td>Public on (.*)</td>')
re.findall(partern, str(ges_info))[0]

'May 25, 2021'

In [56]:
# Title
pattern = re.compile(r'<td style="text-align: justify">(.*)</td>')
detailed_info = re.findall(pattern, str(ges_info))
# title
title_info = detailed_info[0]
# summary
summary_info = detailed_info[1].strip('<br/>')
# over design
design_info = detailed_info[2].strip('<br/>')
# organization name
org_name = detailed_info[3].strip('<br/>')
# city
city = detailed_info[-3]
# country
country = detailed_info[-1]

print(title_info)
print(summary_info)
print(design_info)
print(org_name)
print(city)
print(country)

Tumor-associated hematopoietic stem and progenitor cells positively linked to glioblastoma progression
Brain tumors are typically immunosuppressive and refractory to immunotherapies for reasons that remain poorly understood. The unbiased profiling of immune cell types in the tumor microenvironment may reveal immunologic networks affecting therapy and course of disease. Here we identify and validate the presence of hematopoietic stem and progenitor cells (HSPCs) within tumor tissues that are positively linked to glioblastoma malignancy. In contrast to the medullary hematopoietic compartment, tumor-associated HSPCs are comprised of a majority of CD38- immature cells, such as hematopoietic stem cells and multipotent progenitors, express genes related to glioblastoma progression and display patterns of accelerated cell cycle. When cultured ex vivo, tumor-associated HSPCs form prevalently myeloid colonies, suggesting potential in situ myelopoiesis. In experimental models, HSPCs promote tumo

In [119]:
# Experiment type
pattern = re.compile(r'<td>(.*)<br/></td>')
re.findall(pattern, str(ges_info))[0]

'Expression profiling by high throughput sequencing'

In [74]:
# contributor
contributor = ";".join([i.text for i in ges_info.find_all('a', href=lambda value: value and '/pubmed/?term' in value)])
print(contributor)

Kebir S;Cima I;Dobersalske C


In [99]:
# pmid
ges_info.find('span', {'class': 'pubmed_id'}).get('id')

'34162860'

In [104]:
pattern = re.compile(r'<td>([A-Z][a-z]+ \d{1,2}, \d{4})</td>')
# first time is submission data, second last update date
re.findall(pattern, str(ges_info))

['Jan 21, 2021', 'Jul 14, 2021']

In [107]:
# platform
ges_info.find_all('td', {'valign':'top'})[1].text

'Illumina NovaSeq 6000 (Homo sapiens)'

In [125]:
gsm_list = []
for i in ges_info.find_all('td', {'valign':'top'}):
    gsm = i.find('a', href=lambda value: value and '/geo/query/acc.cgi?acc' in value)
    if "GSM" in str(gsm):
       gsm_list.append(gsm.get('href').strip('/geo/query/acc.cgi?acc='))
print(";".join(gsm_list))

GSM5028734;GSM5028735;GSM5028736;GSM5028737;GSM5028738;GSM5028739;GSM5028740;GSM5028741;GSM5028742;GSM5028743;GSM5028744;GSM5028745;GSM5028746;GSM5028747;GSM5028748;GSM5028749
